<a href="https://colab.research.google.com/github/tonyjosephsebastians/AI-Design-patterns/blob/main/Group_4_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(REST & service boundaries)

Failure
Breaking API changes
Inconsistent responses
Clients tightly coupled to backend logic
Patterns
Resource-Oriented Design
API Versioning Pattern
Structured Error Pattern
Pagination & Filtering Pattern
API Gateway Pattern
Backend-for-Frontend (BFF) Pattern
Proxy Pattern (GoF — auth, cache, rate limiting)
📌 Goal: Stable, evolvable APIs

In [ ]:
!pip -q install fastapi uvicorn httpx pydantic

from fastapi import FastAPI, Query, Header, Request
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Any
import time, uuid


0) The Problem: “Messy API” (Build + Break)
Symptoms you listed (real life):

You change a field name → mobile app breaks

Some endpoints return {"data": ...} others return raw list

Errors are random strings → clients can’t reliably handle them

Clients rely on backend “internal logic” (tight coupling)

We’ll start with a bad API, then fix it using patterns.

In [ ]:
bad_app = FastAPI(title="Bad API")

In [ ]:
@bad_app.get("/user/{user_id}")
def get_user(user_id: int):
    # inconsistent shape: sometimes dict, sometimes list
    if user_id == 1:
        return {"id": 1, "name": "Tony", "age": 30}
    return ["not found"]  # bad

@bad_app.get("/orders")
def list_orders():
    # raw list, no pagination, no metadata
    return [{"id": 101, "total": 99.5}, {"id": 102, "total": 15.0}]

@bad_app.get("/search")
def search(q: str):
    # random error format
    if len(q) < 3:
        return {"error": "too short"}  # not structured
    return {"results": [{"id": 1, "match": q}]}

What breaks?

client expects {id,name} but gets a list

no consistent error handling

no stable evolution path

Now let’s rebuild properly with GROUP 4 patterns.

1) Resource-Oriented Design (ROD)
Idea

Design around resources (nouns) and standard verbs:

GET /users/{id}

GET /users (list)

POST /users

PATCH /users/{id}

GET /users/{id}/orders (sub-resource)

✅ predictable
✅ consistent
✅ easier for clients

In [ ]:
{
  "data": ...,
  "meta": {...},
  "request_id": "..."
}


{'data': Ellipsis, 'meta': {Ellipsis}, 'request_id': '...'}

In [ ]:
app = FastAPI(title="Group 4 - Stable APIs")

In [ ]:
class Meta(BaseModel):
    version: str
    page: Optional[int] = None
    page_size: Optional[int] = None
    total: Optional[int] = None

class Envelope(BaseModel):
    data: Any
    meta: Meta
    request_id: str

FAKE_USERS = {
    1: {"id": 1, "full_name": "Tony Joseph", "email": "tony@example.com"},
    2: {"id": 2, "full_name": "Aleesa", "email": "aleesa@example.com"},
}

@app.middleware("http")
async def add_request_id(request: Request, call_next):
    request.state.request_id = str(uuid.uuid4())
    return await call_next(request)

def ok(data: Any, request_id: str, meta: Optional[Meta] = None):
    return Envelope(data=data, meta=meta or Meta(version="v1"), request_id=request_id)

@app.get("/v1/users/{user_id}", response_model=Envelope)
def get_user_v1(user_id: int, request: Request):
    user = FAKE_USERS.get(user_id)
    if not user:
        # we will fix errors in next section
        return ok({"message": "not found"}, request.state.request_id)
    return ok(user, request.state.request_id)

Now endpoints are resource-shaped and consistent.

2) Structured Error Pattern
Goal

Errors must be machine-readable (clients can switch on code).

Recommended structure:

In [ ]:
{
  "error": {
    "code": "USER_NOT_FOUND",
    "message": "User 999 not found",
    "details": {...}
  },
  "request_id": "..."
}


{'error': {'code': 'USER_NOT_FOUND',
  'message': 'User 999 not found',
  'details': {Ellipsis}},
 'request_id': '...'}

Add Structured Errors Globally

In [ ]:
class ErrorBody(BaseModel):
    code: str
    message: str
    details: Dict[str, Any] = Field(default_factory=dict)

class ErrorEnvelope(BaseModel):
    error: ErrorBody
    request_id: str

class ApiError(Exception):
    def __init__(self, code: str, message: str, status: int = 400, details=None):
        self.code = code
        self.message = message
        self.status = status
        self.details = details or {}

@app.exception_handler(ApiError)
def api_error_handler(request: Request, exc: ApiError):
    return JSONResponse(
        status_code=exc.status,
        content=ErrorEnvelope(
            error=ErrorBody(code=exc.code, message=exc.message, details=exc.details),
            request_id=request.state.request_id
        ).model_dump()
    )

@app.get("/v1/users/{user_id}", response_model=Envelope)
def get_user_v1(user_id: int, request: Request):
    user = FAKE_USERS.get(user_id)
    if not user:
        raise ApiError(
            code="USER_NOT_FOUND",
            message=f"User {user_id} not found",
            status=404,
            details={"user_id": user_id}
        )
    return ok(user, request.state.request_id)


✅ Now every error looks the same.
✅ Clients can reliably handle code.

3) API Versioning Pattern
Why?

Because your API will evolve:

rename full_name → name

add fields

change behavior without breaking old clients

Practical approach

URL versioning: /v1/..., /v2/... (simple, very common)

or header versioning: Accept: application/vnd.app.v2+json

We’ll do URL versioning.

In [ ]:
@app.get("/v2/users/{user_id}", response_model=Envelope)
def get_user_v2(user_id: int, request: Request):
    user = FAKE_USERS.get(user_id)
    if not user:
        raise ApiError("USER_NOT_FOUND", f"User {user_id} not found", 404, {"user_id": user_id})

    # breaking change: field renamed full_name -> name
    v2_user = {"id": user["id"], "name": user["full_name"], "email": user["email"]}
    return ok(v2_user, request.state.request_id, meta=Meta(version="v2"))


✅ Old apps keep using /v1
✅ New apps upgrade to /v2
✅ You can deprecate /v1 later with a timeline

Pagination & Filtering Pattern
Problem

Without pagination:

endpoints become slow

clients download too much

you can’t scale

Standard pattern

GET /v1/users?page=1&page_size=20&sort=name&filter=email:example.com

In [ ]:
@app.get("/v1/users", response_model=Envelope)
def list_users_v1(
    request: Request,
    page: int = Query(1, ge=1),
    page_size: int = Query(2, ge=1, le=50),
    email_contains: Optional[str] = None
):
    users = list(FAKE_USERS.values())

    if email_contains:
        users = [u for u in users if email_contains.lower() in u["email"].lower()]

    total = len(users)
    start = (page - 1) * page_size
    end = start + page_size
    page_items = users[start:end]

    meta = Meta(version="v1", page=page, page_size=page_size, total=total)
    return ok(page_items, request.state.request_id, meta=meta)


✅ predictable list API
✅ clients can “infinite scroll”
✅ backend stays fast

API Gateway Pattern
What it is

A gateway sits in front of services:

auth (JWT, SSO)

rate limiting

routing /users → user-service, /orders → order-service

request/response normalization

logging/tracing

In Colab terms

We can simulate a “gateway” as another FastAPI app that forwards calls.

In [ ]:
gateway = FastAPI(title="Gateway")
def require_api_key(x_api_key: Optional[str]):
    if x_api_key != "secret123":
        raise ApiError("UNAUTHORIZED", "Invalid API key", 401)

@gateway.get("/api/users/{user_id}", response_model=Envelope)
def gateway_get_user(user_id: int, request: Request, x_api_key: Optional[str] = Header(None)):
    require_api_key(x_api_key)
    # In real life: forward to user-service
    return get_user_v1(user_id, request)

✅ Clients talk to one place
✅ Services can change internally without clients knowing

Backend-for-Frontend (BFF) Pattern
When you need it

Mobile app wants:

small payloads

aggregated fields
Web app wants:

richer payloads
If both hit the same backend directly → API becomes a mess.

BFF solution

Create:

mobile-bff

web-bff

Each one reshapes data for that frontend.

In [ ]:
mobile_bff = FastAPI(title="Mobile BFF")

@mobile_bff.get("/mobile/users/{user_id}", response_model=Envelope)
def mobile_user(user_id: int, request: Request):
    user = FAKE_USERS.get(user_id)
    if not user:
        raise ApiError("USER_NOT_FOUND", f"User {user_id} not found", 404)

    mobile_shape = {"id": user["id"], "name": user["full_name"]}  # email removed
    return ok(mobile_shape, request.state.request_id, meta=Meta(version="mobile-v1"))


✅ Mobile app is stable
✅ Web app can still get full details elsewhere
✅ You stop “one API to serve all” chaos

Proxy Pattern (GoF) — auth/cache/rate-limit wrapper

Proxy wraps access to a service:

check auth

cache response

throttle requests

add retry

In Python, you can implement proxy as a function/class around your “real client”.

In [ ]:
class UserServiceClient:
    def get_user(self, user_id: int) -> dict:
        user = FAKE_USERS.get(user_id)
        if not user:
            raise ApiError("USER_NOT_FOUND", f"User {user_id} not found", 404)
        return user

class UserServiceProxy:
    def __init__(self, real: UserServiceClient):
        self.real = real
        self.cache = {}
        self.calls = []  # timestamps

    def rate_limit(self, per_sec=5):
        now = time.time()
        self.calls = [t for t in self.calls if now - t < 1]
        if len(self.calls) >= per_sec:
            raise ApiError("RATE_LIMITED", "Too many requests", 429)
        self.calls.append(now)

    def get_user(self, user_id: int) -> dict:
        self.rate_limit()
        if user_id in self.cache:
            return self.cache[user_id]
        data = self.real.get_user(user_id)
        self.cache[user_id] = data
        return data

proxy = UserServiceProxy(UserServiceClient())

@app.get("/v1/proxy/users/{user_id}", response_model=Envelope)
def proxy_user(user_id: int, request: Request):
    return ok(proxy.get_user(user_id), request.state.request_id, meta=Meta(version="v1"))


✅ adds cache
✅ adds rate limit
✅ doesn’t change core service logic

✅ Resource-oriented endpoints (nouns, predictable routes)
✅ Consistent response envelope
✅ Structured errors with stable code
✅ Versioning strategy (URL /v1, /v2)
✅ Pagination + filtering for list endpoints
✅ Gateway for cross-cutting concerns
✅ BFF when multiple frontends need different shapes
✅ Proxy for auth/cache/rate-limit wrappers